In [1]:
import types, sys, IPython, traceback, ipykernel, traitlets, operator, toolz.curried as toolz, tokenize, io, textwrap

ip = IPython.get_ipython()


In [2]:
    ip = get_ipython()

In [1]:
    def get_inline_expression(string, pos):
        try:
            line, offset = IPython.utils.tokenutil.line_at_cursor(string, pos)
        except tokenize.TokenError: return 
        inline = True
        snippet, token, line = '', '`', ''
        while (offset < pos) and line:
            inline = not inline
            snippet, token, line = line.partition('`')
            offset += len(snippet) + len(token)
        return inline and snippet or None

In [2]:
    class PidginKernel(type(ip.kernel)):        
        def do_inspect(self, code, cursor_pos, detail_level=0):
            inspect = False
            if (cursor_pos-2)>0 and cursor_pos-1 <= len(code) :
                if code[cursor_pos-1] == '?':
                    inspect = True
                    if code[cursor_pos-2] == '?':
                        detail_level = 1

            if not inspect and '`' in code:
                inline = get_inline_expression(code, cursor_pos)
                if inline:
                    return dict(self.shell.sync_user_expression(inline), found=True)

            result = ipykernel.ipkernel.IPythonKernel.do_inspect(self, code, cursor_pos, detail_level)
            
            if not result['found']:
                result['found'] = True
                result['data'].update({'text/markdown': code})
            return result

NameError: name 'ip' is not defined

In [5]:
    original_methods = {}
    def load_ipython_extension(ip):
        global original_methods
        original_methods['do_inspect'] = original_methods.get(
            'do_inspect', ip.kernel.do_inspect)
        ip.kernel.do_inspect = types.MethodType(PidginKernel.do_inspect, ip.kernel)

    def unload_ipython_extension(ip): 
        ip.kernel.__dict__.update(original_methods)

In [6]:
    if __name__ == '__main__':
        #!ipython -m pytest -- shell.ipynb
        %reload_ext pidgin.shell
        load_ipython_extension(get_ipython())